In [1]:
from ogb.nodeproppred import NodePropPredDataset
from ogb.graphproppred import GraphPropPredDataset
from sklearn.model_selection import ParameterGrid
%reload_ext autoreload
%autoreload 2
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import convo as c 
import convL 
import modularized as m 
import batching as b 
import poolingL
import gin
import math 
import funcy as fy
import gridSearch as g 
import gridS
import json 
import os



/home/olga/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-07 14:53:25.619698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 14:53:29.331622: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-07 14:53:29.332013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [2]:
dataset1 = GraphPropPredDataset(name = 'ogbg-molhiv')

In [3]:
dataset_arxiv = NodePropPredDataset(name = 'ogbn-arxiv')

In [4]:
dataset_arxiv.get_idx_split()

{'train': array([     0,      1,      2, ..., 169145, 169148, 169251]),
 'valid': array([   349,    357,    366, ..., 169185, 169261, 169296]),
 'test': array([   346,    398,    451, ..., 169340, 169341, 169342])}

In [5]:
param_grid = {'num_layers': [1, 2, 3, 4, 5, 6],
              'learning_rate': [0.0001, 0.001, 0.01],
              'optimizer': ['Adam'],
              'regularization': ['DropOut', 'NodeSampling', 'DropEdge', 'GDC'],
              'probability': [0.3, 0.5, 0.7],
              'activation': ['relu', 'sigmoid', 'tanh'],
              'units': [32, 64],
              'convo_type': ['gcn', 'gin']}

override_conv_type = 'gin' # None

if override_conv_type is not None:
    param_grid['convo_type'] = [override_conv_type]

small_sample = list(ParameterGrid(param_grid))[0:1]
sample = list(ParameterGrid(param_grid))[0:5]
big_sample = list(ParameterGrid(param_grid))[0:20]
small_sample

[{'activation': 'relu',
  'convo_type': 'gin',
  'learning_rate': 0.0001,
  'num_layers': 1,
  'optimizer': 'Adam',
  'probability': 0.3,
  'regularization': 'DropOut',
  'units': 32}]

In [6]:
ds = GraphPropPredDataset('ogbg-molhiv')

In [7]:
test = b.make_tf_datasets(ds)['train']

test

2023-05-06 17:16:16.406337: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-06 17:16:16.406888: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-06 17:16:16.406919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-UCI9MJD): /proc/driver/nvidia/version does not exist
2023-05-06 17:16:16.410889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<FlatMapDataset element_spec=({'X': TensorSpec(shape=(None, 9), dtype=tf.float32, name=None), 'ref_A': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), 'ref_B': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), 'num_nodes': TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [8]:
g.create_grid_models(big_sample,'ogbg-molhiv')

Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropOut units:32 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropOut units:64 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:NodeSampling units:32 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:NodeSampling units:64 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropEdge units:32 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer:Adam probability:0.3 regularization:DropEdge units:64 
Evaluating hpconfig activation:relu convo_type:gin learning_rate:0.0001 num_layers:1 optimizer

1

In [53]:
gridS.train_and_evaluate(small_sample, 'ogbg-molhiv')

Evaluating hyperparameter configuration 0 for dataset ogbg-molhiv.
138/138 [==============================] - 1s 3ms/step


NameError: name 'test_acc' is not defined

In [17]:
g.create_grid_models(sample, 'ogbn-arxiv')


IndexError: index 1 is out of bounds for axis 0 with size 1

In [9]:
from ogb.nodeproppred import Evaluator
from ogb.graphproppred import Evaluator
evaluator = Evaluator(name = 'ogbg-molhiv')
evaluator


In [10]:
node_features = GraphPropPredDataset(name='ogbg-molhiv')
test_data = b.make_tf_datasets(node_features)['test']


In [16]:
g.config('ogbg-molhiv')

{'include_mask': False,
 'num_tasks': 1,
 'loss': <keras.losses.BinaryCrossentropy at 0x7f6638fb0310>,
 'metrics': [<keras.metrics.metrics.AUC at 0x7f6638fb0520>,
 'evaluator': <ogb.graphproppred.evaluate.Evaluator at 0x7f6638f2ae30>}

In [18]:
current_directory = os.getcwd()
current_directory

type(sample) 

for entry in sample: 
    print(entry.keys())

dict_keys(['activation', 'convo_type', 'learning_rate', 'num_layers', 'optimizer', 'probability', 'regularization', 'units'])
dict_keys(['activation', 'convo_type', 'learning_rate', 'num_layers', 'optimizer', 'probability', 'regularization', 'units'])
dict_keys(['activation', 'convo_type', 'learning_rate', 'num_layers', 'optimizer', 'probability', 'regularization', 'units'])
dict_keys(['activation', 'convo_type', 'learning_rate', 'num_layers', 'optimizer', 'probability', 'regularization', 'units'])
dict_keys(['activation', 'convo_type', 'learning_rate', 'num_layers', 'optimizer', 'probability', 'regularization', 'units'])


In [29]:
for entry in sample:
    for key, value in entry.items():
        with open(key + ".json", "w") as file:
            json.dump(value, file)
